<h1 align=center>Segmenting and Clustering Neighborhoods in Toronto</h1>

<br>

In this assignment from Coursera, I will explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. 

I will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset. Once the data is in a structured format, I can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

i)   The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood<br>
ii)  Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.<br>
iii) More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.<br>
iv)  If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.<br>
v)   Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.<br>
vi)  In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

The code will be built to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

<br>

<h2>1. First part of the task</h2>

First of all, let us import the necessary libraries

In [21]:
import pandas as pd #for data analysis
import numpy as np #to handle data in a vectorized manner
from bs4 import BeautifulSoup #library for web scraping
import requests #to manage requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Getting the data

In [22]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')
table = soup.table

Finding data from tag and storing it in a list

In [23]:
#Reminding the first requirement: "the dataframe will consist of three columns: PostalCode, Borough, and Neighborhood"
postcode = []
borough = []
neighborhood = []

for cell in table.find_all('td'):
    postcode.append(cell.b.text)
    borough.append(cell.span.text.split('(')[0])
    try:
        neigh = cell.span.text
        #split neighborhood by brackets and repace / with ,
        neighbor = ''.join(neigh.split('(')[1].split(')'))
        neighborhood.append(neighbor.replace('/',','))  
    except Exception as e:
        neighborhood.append('Not assigned')

**Converting data into Dataframe**

In [24]:
df = pd.DataFrame(list(zip(postcode,borough,neighborhood)),columns=['PostalCode','Borough','Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"


Time to clean data. "Ignore cells with a borough that is Not assigned."

In [26]:
#how many "not assigned" values?
len(df[df['Borough'] == 'Not assigned'])

77

In [27]:
#dropping rows
df = df.drop(df[df['Borough'] == 'Not assigned'].index)
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Queen's Park / Ontario Provincial Government,Not assigned


In [28]:
#check again
len(df[df['Borough'] == 'Not assigned'])

0

In [29]:
df.shape

(103, 3)

Now, "if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough."

In [30]:
df[df['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood
6,M7A,Queen's Park / Ontario Provincial Government,Not assigned


In [31]:
#replacing
df.loc[df['Neighborhood'] == 'Not Assigned', 'Neighborhood'] = df['Borough']

In [32]:
df[df['Neighborhood'] == 'Not Assigned'][0:5]

,PostalCode,Borough,Neighborhood


In [33]:
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Queen's Park / Ontario Provincial Government,Not assigned


Reminding the last requirement, I will use the shape method to print the number of rows of the dataframe.

In [34]:

df.shape

(103, 3)

#   

<h2>2. Second part of the task</h2>

In order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood

In [35]:
df_lat = pd.read_csv('http://cocl.us/Geospatial_data')
df_lat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Changing the column "Postal Code" to "PostalCode" to merge the two data frames together

In [36]:
df_lat.columns = ['PostalCode','Latitude','Longitude']
df_lat.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging both dataframe

In [38]:
df_can = pd.merge(df,df_lat, on='PostalCode')
df_can.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park / Ontario Provincial Government,Not assigned,43.662301,-79.389494


In [39]:
df.shape

(103, 3)

<br>

<h2>3. Third part of the task</h2>

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data.

In [60]:
import json #to handle JSON files
from pandas.io.json import json_normalize #to tranform JSON file into a dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Nidria\Anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.0.1               |             py_0         575 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    certifi-2019.9.11          |           py37_0         147 KB  conda-forge
    conda-4.8.3                |   py37hc8dfbb8_1         3.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    -------------------------------------------------



==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda




Libraries imported.


How many boroughs and neighborhoos there are?

In [61]:
print("The dataframe has {} Borough and {} Neighborhood".format(len(df_can['Borough'].unique()),len(df_can['Neighborhood'])))

The dataframe has 15 Borough and 103 Neighborhood


I need to install geopy in order to get the latitude and longitude of Toronto

In [69]:
!pip install geopy

In [70]:
from geopy.geocoders import Nominatim
address = "Toronto, Ontario"
geolocator = Nominatim(user_agent='to_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The coordinates of Toronto are {} , {}".format(latitude,longitude))

The coordinates of Toronto are 43.653963 , -79.387207


Generating the map to visualize the neighborhoods

In [71]:
toronto_map = folium.Map(
    location = [latitude,longitude],
    zoom_start = 11
)

for lat,lng,borough,neighborhood in zip(df_can['Latitude'],df_can['Longitude'],df_can['Borough'],df_can['Neighborhood']):
    label = '{}, {}'.format(neighborhood,borough)
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius = 10,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        popup = label
    ).add_to(toronto_map)
    
toronto_map

Declaring Foursquare's credentials

In [98]:
LIMIT = 100
CLIENT_ID = 'HP3ZXZQM3CXI0LS4I5SIF2JHKLENDJG1GTESFNCTJC5RAWXA' # your Foursquare ID
CLIENT_SECRET = '2NFNYJYUAZFL14T1KM1RFBDH5NKC5EEKPMX41ORTTH5DQLUH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HP3ZXZQM3CXI0LS4I5SIF2JHKLENDJG1GTESFNCTJC5RAWXA
CLIENT_SECRET:2NFNYJYUAZFL14T1KM1RFBDH5NKC5EEKPMX41ORTTH5DQLUH


Exploring the neighborhood

In [73]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        #this is the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        #GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        #return information for each nearby venue such as name, location or category
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [75]:
toronto_venues = getNearbyVenues(names=df_can['Neighborhood'],latitudes=df_can['Latitude'],longitudes=df_can['Longitude'])

Taking the previous information into a new dataset

In [76]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [77]:
toronto_venues.shape

(2233, 7)

How many nevues there are for each neighborhood?

In [78]:
neigh_venue = toronto_venues.groupby('Neighborhood')['Venue'].count()
neigh_venue.to_frame()[0:5]

,Venue
Neighborhood,
Agincourt,5
"Alderwood , Long Branch",11
"Bathurst Manor , Wilson Heights , Downsview North",21
Bayview Village,4
"Bedford Park , Lawrence Manor East",25


In [79]:
print('There are {} unique venue categories'.format(len(toronto_venues['Venue Category'].unique())))

There are 271 unique venue categories


## Analyze Each Neighborhood

Dropping the venue category neighborhood

In [80]:
toronto_venues = toronto_venues.drop(toronto_venues[toronto_venues['Venue Category'] == 'Neighborhood'].index)

One Hot Enconding

In [81]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']],prefix="", prefix_sep="")

toronto_onehot = pd.concat([toronto_venues['Neighborhood'],toronto_onehot],axis=1) # merge neighborhood column back to dataframe

toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [82]:
toronto_onehot.shape

(2229, 271)

Grouping rows according to the neighborhood and mean of the ocurrence of each category

In [83]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000

In [84]:
toronto_grouped.shape

(99, 271)

Printing each neighborhood with the 5 most common venues

In [85]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt ----
                       venue  freq
0             Clothing Store   0.2
1             Breakfast Spot   0.2
2                     Lounge   0.2
3               Skating Rink   0.2
4  Latin American Restaurant   0.2


----Alderwood , Long Branch----
                venue  freq
0         Pizza Place  0.18
1        Skating Rink  0.09
2      Sandwich Place  0.09
3  Athletics & Sports  0.09
4        Dance Studio  0.09


----Bathurst Manor , Wilson Heights , Downsview North----
           venue  freq
0           Bank  0.10
1    Coffee Shop  0.10
2       Pharmacy  0.05
3      Gift Shop  0.05
4  Shopping Mall  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4          Music Venue  0.00


----Bedford Park , Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.12
1      Sandwich Place  0.08
2          Restaurant  0.08
3         Coff



----Kensington Market , Chinatown , Grange Park----
                           venue  freq
0                            Bar  0.07
1                           Café  0.06
2          Vietnamese Restaurant  0.06
3  Vegetarian / Vegan Restaurant  0.05
4                         Bakery  0.05


----Kingsview Village , St. Phillips , Martin Grove Gardens , Richview Gardens----
               venue  freq
0        Pizza Place  0.25
1  Mobile Phone Shop  0.25
2           Bus Line  0.25
3     Sandwich Place  0.25
4        Men's Store  0.00


----Lawrence Manor , Lawrence Heights----
                    venue  freq
0          Clothing Store  0.23
1  Furniture / Home Store  0.15
2       Accessories Store  0.08
3             Coffee Shop  0.08
4             Event Space  0.08


----Lawrence Park----
                venue  freq
0  Photography Studio  0.25
1            Bus Line  0.25
2                Park  0.25
3         Swim School  0.25
4   Accessories Store  0.00


----Leaside----
                   

4  Miscellaneous Shop  0.00




Creating a new dataframe with the previous information

In [86]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [87]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Clothing Store,Breakfast Spot,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,"Alderwood , Long Branch",Pizza Place,Gym,Coffee Shop,Sandwich Place,Athletics & Sports,Pub,Pool,Skating Rink,Pharmacy,Dance Studio
2,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Pharmacy,Frozen Yogurt Shop,Bridal Shop,Sandwich Place,Diner,Restaurant,Middle Eastern Restaurant,Supermarket
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,"Bedford Park , Lawrence Manor East",Italian Restaurant,Sandwich Place,Restaurant,Coffee Shop,Pharmacy,Cupcake Shop,Juice Bar,Fast Food Restaurant,Butcher,Indian Restaurant


Clustering Neighborhoods

In [88]:
#k-means algorithm to cluster the neighborhood into 5 clusters

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',axis=1)

kmeans = KMeans(n_clusters = kclusters, random_state=0 ).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 4])

New dataframe includes with the top 10 venues for each neighborhood

In [89]:
neighborhoods_venues_sorted.insert(1,'Cluster Labels', kmeans.labels_)

Merging toronto_grouped with toronto_data to add latitude/longitude for each neighborhood

In [90]:
toronto_merged = df_can.copy()

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Park,Food & Drink Shop,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Hockey Arena,French Restaurant,Coffee Shop,Portuguese Restaurant,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,0.0,Coffee Shop,Bakery,Pub,Park,Theater,Restaurant,Café,Mexican Restaurant,French Restaurant,Chocolate Shop
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,0.0,Clothing Store,Furniture / Home Store,Accessories Store,Athletics & Sports,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Women's Store,Vietnamese Restaurant
4,M7A,Queen's Park / Ontario Provincial Government,Not assigned,43.662301,-79.389494,0.0,Coffee Shop,Park,Yoga Studio,Nightclub,Beer Bar,Boutique,Seafood Restaurant,Sandwich Place,Burger Joint,Burrito Place


Time to visualize

In [91]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11) # creating the map

markers_colors = [] #defining properties
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examining first cluster

In [92]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0.0,Hockey Arena,French Restaurant,Coffee Shop,Portuguese Restaurant,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
2,Downtown Toronto,0.0,Coffee Shop,Bakery,Pub,Park,Theater,Restaurant,Café,Mexican Restaurant,French Restaurant,Chocolate Shop
3,North York,0.0,Clothing Store,Furniture / Home Store,Accessories Store,Athletics & Sports,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Women's Store,Vietnamese Restaurant
4,Queen's Park / Ontario Provincial Government,0.0,Coffee Shop,Park,Yoga Studio,Nightclub,Beer Bar,Boutique,Seafood Restaurant,Sandwich Place,Burger Joint,Burrito Place
6,Scarborough,0.0,Fast Food Restaurant,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
7,North York,0.0,Café,Gym / Fitness Center,Caribbean Restaurant,Japanese Restaurant,Baseball Field,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
8,East York,0.0,Pizza Place,Athletics & Sports,Gastropub,Intersection,Fast Food Restaurant,Pet Store,Pharmacy,Bus Line,Breakfast Spot,Bank
9,Downtown Toronto,0.0,Coffee Shop,Clothing Store,Japanese Restaurant,Café,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant,Restaurant,Bookstore,Diner
10,North York,0.0,Pizza Place,Pub,Park,Asian Restaurant,Japanese Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
11,Etobicoke,0.0,Golf Course,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


Examining second cluster

In [93]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
98,Etobicoke,1.0,River,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Department Store


Examining third cluster

In [94]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,North York,2.0,Pizza Place,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Falafel Restaurant
70,Etobicoke,2.0,Pizza Place,Middle Eastern Restaurant,Sandwich Place,Coffee Shop,Discount Store,Chinese Restaurant,Dim Sum Restaurant,Diner,Distribution Center,Dog Run
77,Etobicoke,2.0,Pizza Place,Sandwich Place,Bus Line,Mobile Phone Shop,Distribution Center,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Ethiopian Restaurant


Examining fourth cluster

In [95]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,3.0,Cafeteria,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio,College Rec Center


Examining fifth cluster

In [96]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,4.0,Park,Food & Drink Shop,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
21,York,4.0,Park,Women's Store,Market,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant
35,East YorkEast Toronto,4.0,Metro Station,Park,Convenience Store,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
40,North York,4.0,Airport,Park,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
66,North York,4.0,Park,Bank,Convenience Store,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
83,Central Toronto,4.0,Park,Trail,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
85,Scarborough,4.0,Playground,Park,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
91,Downtown Toronto,4.0,Park,Playground,Trail,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center


<h4>Conclusion</h4>

Biggest clusters: 1, 3 and 5<br>
One neighborhood per cluster: 2 and 4